# Journey North Egg Sightings

In [1]:
#  Import and read
import pandas as pd 
import numpy as np
import json
# Sue Mottet 2-19-2022

# geoJSONifier

In [2]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

# Load file

In [3]:
JNorth1996_2020_df = pd.read_csv("UWMadisonArb_JNorth_Project_MonarchsAndMilkweed_1996-2020.csv")
JNorth1996_2020_df.head()

C:\Users\motte\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,sighting_date,species,number,latitude,longitude,flag_location,comments,school,grade,image_url,customerid,number_old,flag_number,species_old,flag_species
0,850841703,1996-12-16,Monarch Adult Sighted,1,27.594,-81.362,0,On the ballfield a solitary monarch butterfly ...,Walker memorial junior academy,10.0,-999999.999,135,-999999.999,1,Monarch Butterfly,1
1,851824801,1996-12-06,Monarch Adult Sighted,1,30.420,-87.224,0,Our 5th grade Discovery students who went alon...,Hambrick elementary school,5.0,-999999.999,150,-999999.999,1,Monarch Butterfly,1
2,854409931,1996-09-21,Monarch Adult Sighted,1,40.817,-96.710,0,We saw several hundred monarchs in our school ...,-999999.999,5.0,-999999.999,9701054,-999999.999,1,Monarch Butterfly,1
3,854557206,1996-09-21,Monarch Adult Sighted,1,40.817,-96.710,0,Over a hundered monarchs in our school butterf...,Fredstrom elementary school,6.0,-999999.999,175,-999999.999,1,Monarch Butterfly,1
4,855061862,1997-01-01,Monarch Adult Sighted,1,34.244,-77.946,0,"I Thought I Saw It, But I Was Suprised To See ...",Gregory,3.0,-999999.999,181,-999999.999,1,Monarch Butterfly,1


# Data Exploratory

In [4]:
JNorth1996_2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227433 entries, 0 to 227432
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             227433 non-null  int64  
 1   sighting_date  227433 non-null  object 
 2   species        227433 non-null  object 
 3   number         227433 non-null  int64  
 4   latitude       227433 non-null  float64
 5   longitude      227433 non-null  float64
 6   flag_location  227433 non-null  int64  
 7   comments       227433 non-null  object 
 8   school         227433 non-null  object 
 9   grade          227433 non-null  float64
 10  image_url      227433 non-null  object 
 11  customerid     227433 non-null  int64  
 12  number_old     227433 non-null  float64
 13  flag_number    227433 non-null  int64  
 14  species_old    227433 non-null  object 
 15  flag_species   227433 non-null  int64  
dtypes: float64(4), int64(6), object(6)
memory usage: 27.8+ MB


In [5]:
str(JNorth1996_2020_df.columns.tolist())

"['id', 'sighting_date', 'species', 'number', 'latitude', 'longitude', 'flag_location', 'comments', 'school', 'grade', 'image_url', 'customerid', 'number_old', 'flag_number', 'species_old', 'flag_species']"

In [6]:
JNorth1996_2020_df.describe()

,id,number,latitude,longitude,flag_location,grade,customerid,number_old,flag_number,flag_species
count,2.274330e+05,2.274330e+05,227433.000000,227433.000000,227433.000000,227433.000000,2.274330e+05,2.274330e+05,227433.000000,227433.000000
mean,1.420650e+09,1.186529e+02,38.400690,-87.569965,0.598018,-907386.217597,9.451737e+06,-2.844270e+05,0.294816,0.130324
std,1.456887e+08,2.209080e+04,5.268651,10.880362,0.490299,289894.342744,2.193961e+06,4.518136e+05,0.467089,0.336660
min,8.508417e+08,-1.700000e+01,-12.100000,-165.107000,0.000000,-999999.999000,0.000000e+00,-1.000000e+06,0.000000,0.000000
25%,1.377751e+09,1.000000e+00,34.723000,-95.268000,0.000000,-999999.999000,9.941050e+06,-1.000000e+06,0.000000,0.000000
50%,1.477448e+09,1.000000e+00,39.503000,-86.787000,1.000000,-999999.999000,9.977663e+06,1.000000e+00,0.000000,0.000000
75%,1.526383e+09,3.000000e+00,42.409000,-79.255000,1.000000,-999999.999000,1.000029e+07,3.000000e+00,1.000000,0.000000
max,1.526525e+09,1.000000e+07,65.136000,-52.883000,1.000000,1112.000000,1.004256e+07,1.000000e+07,2.000000,1.000000


In [7]:
JNorth1996_2020_df.columns.tolist()

['id',
 'sighting_date',
 'species',
 'number',
 'latitude',
 'longitude',
 'flag_location',
 'comments',
 'school',
 'grade',
 'image_url',
 'customerid',
 'number_old',
 'flag_number',
 'species_old',
 'flag_species']

In [8]:
JNorth1996_2020_df.shape

(227433, 16)

In [9]:
# Determine the number of unique values in each column.
for col in JNorth1996_2020_df:
  print(JNorth1996_2020_df[col].unique())

[ 850841703  851824801  854409931 ... 1526524404 1526524405 1526525021]
['1996-12-16' '1996-12-06' '1996-09-21' ... '2020-12-29' '2020-12-30'
 '2020-12-31']
['Monarch Adult Sighted' 'Monarch Adult (FIRST sighted)'
 'Monarch Egg (FIRST sighted)' 'Monarch (OTHER Observations)'
 'Monarch Larva (FIRST sighted)' 'Monarch PEAK Migration'
 'Milkweed (FIRST sighted)' 'Monarch Fall Roost' 'Monarch Egg Sighted'
 'Monarch Larva Sighted' 'Monarch, Captive-Reared' 'Milkweed Sighted']
[       1      150       20       89       30     2000       50     7500
       23      200        5      300        4      100       71      120
      325       21       28     5000    20000       36       25        6
       54     1000      160       24       60       37        2     3000
  1000000       10  2000000       40       85       65       15        8
      500       14      112       75       80    15000       18    10000
      912   100000      400      800       12       35      600       16
    25000    

[     135      150  9701054 ... 10042506 10042509 10042557]
[-9.99999999e+05  0.00000000e+00  1.50000000e+02  2.00000000e+01
  8.90000000e+01  3.00000000e+01  2.00000000e+03  5.00000000e+01
  7.50000000e+03  2.30000000e+01  2.00000000e+02  5.00000000e+00
  3.00000000e+02  4.00000000e+00  1.00000000e+02  7.10000000e+01
  1.20000000e+02  3.25000000e+02  2.10000000e+01  2.80000000e+01
  5.00000000e+03  2.00000000e+04  3.60000000e+01  2.50000000e+01
  6.00000000e+00  5.40000000e+01  1.00000000e+03  1.60000000e+02
  2.40000000e+01  6.00000000e+01  3.70000000e+01  2.00000000e+00
  3.00000000e+03  1.00000000e+06  1.00000000e+01  1.00000000e+00
  2.00000000e+06  4.00000000e+01  8.50000000e+01  6.50000000e+01
  1.50000000e+01  8.00000000e+00  5.00000000e+02  1.40000000e+01
  1.12000000e+02  7.50000000e+01  8.00000000e+01  1.50000000e+04
  1.80000000e+01  1.00000000e+04  9.12000000e+02  1.00000000e+05
  4.00000000e+02  8.00000000e+02  1.20000000e+01  3.50000000e+01
  6.00000000e+02  1.60000000e+

In [10]:
# Determine the number of unique values in each column.
for col in JNorth1996_2020_df:
  print(JNorth1996_2020_df[col].value_counts())

850841703     1
1526350827    1
1526350816    1
1526350817    1
1526350818    1
             ..
1432512422    1
1432519704    1
1432562236    1
1432562641    1
1526525021    1
Name: id, Length: 227433, dtype: int64
2018-08-22    392
2018-10-03    377
2018-09-06    353
2019-09-19    337
2018-09-12    335
             ... 
2015-12-31      1
2015-11-24      1
2015-11-22      1
2006-07-30      1
1996-12-16      1
Name: sighting_date, Length: 7491, dtype: int64
Monarch Adult Sighted            102306
Monarch Adult (FIRST sighted)     36409
Monarch Larva Sighted             17378
Monarch, Captive-Reared           14339
Monarch (OTHER Observations)      13581
Monarch Egg Sighted                8736
Milkweed (FIRST sighted)           8465
Monarch PEAK Migration             7689
Monarch Egg (FIRST sighted)        6876
Monarch Fall Roost                 5035
Monarch Larva (FIRST sighted)      4898
Milkweed Sighted                   1721
Name: species, dtype: int64
1       139319
2        24169
3

In [11]:
# Find null values
for column in JNorth1996_2020_df.columns:
   print(f"Column {column} has {JNorth1996_2020_df[column].isnull().sum()} null values")

Column id has 0 null values
Column sighting_date has 0 null values
Column species has 0 null values
Column number has 0 null values
Column latitude has 0 null values
Column longitude has 0 null values
Column flag_location has 0 null values
Column comments has 0 null values
Column school has 0 null values
Column grade has 0 null values
Column image_url has 0 null values
Column customerid has 0 null values
Column number_old has 0 null values
Column flag_number has 0 null values
Column species_old has 0 null values
Column flag_species has 0 null values


In [12]:
# Find duplicate entries
print(f"Duplicate entries: {JNorth1996_2020_df.duplicated().sum()}")

Duplicate entries: 0


In [13]:
# filtering by type of observation (Monarch Egg Sighted)
JNorth_Egg_Site_df = JNorth1996_2020_df[JNorth1996_2020_df['species']=='Monarch Egg Sighted']
JNorth_Egg_Site_df.head()

,id,sighting_date,species,number,latitude,longitude,flag_location,comments,school,grade,image_url,customerid,number_old,flag_number,species_old,flag_species
48263,1343918156,2012-08-02,Monarch Egg Sighted,4,39.091,-76.624,0,Eggs on common mw in open field.,Arlington echo outdoor education center,-999999.999,-999999.999,9954266,4.000,0,Monarch Egg Sighted,0
48269,1344034477,2012-08-02,Monarch Egg Sighted,1,42.647,-74.575,0,"Collected 2 eggs today, feel as if I&#39;m off...",-999999.999,-999999.999,-999999.999,9940596,-999999.999,1,Monarch Egg Sighted,0
48286,1344379065,2012-08-07,Monarch Egg Sighted,2,42.647,-74.575,0,2 eggs collected.,-999999.999,-999999.999,-999999.999,9940596,2.000,0,Monarch Egg Sighted,0
48294,1344447589,2012-08-05,Monarch Egg Sighted,1,32.774,-91.709,0,female has started laying eggs :) found about...,-999999.999,-999999.999,https://journeynorth.org/jnorth/images/userfil...,9949176,-999999.999,1,Monarch Egg Sighted,0
48298,1344457901,2012-08-08,Monarch Egg Sighted,1,42.647,-74.575,0,2 eggs collected.,-999999.999,-999999.999,-999999.999,9940596,-999999.999,1,Monarch Egg Sighted,0


In [14]:
JNorth_Egg_Site_df.to_csv('JNorth_Egg_Site.csv', header=True, mode='w' )

In [15]:
JNorth_Egg_Site_df.shape

(8736, 16)

# Format file for geoJSON 

In [16]:
JNorth_Egg_Site_df = pd.DataFrame(JNorth_Egg_Site_df)
JNorth_Egg_Site_df.drop(['id','flag_location', 'comments','school','grade','image_url','customerid','number_old','flag_number','species_old','flag_species'], axis=1, inplace=True)
JNorth_Egg_Site_df.head()

,sighting_date,species,number,latitude,longitude
48263,2012-08-02,Monarch Egg Sighted,4,39.091,-76.624
48269,2012-08-02,Monarch Egg Sighted,1,42.647,-74.575
48286,2012-08-07,Monarch Egg Sighted,2,42.647,-74.575
48294,2012-08-05,Monarch Egg Sighted,1,32.774,-91.709
48298,2012-08-08,Monarch Egg Sighted,1,42.647,-74.575


In [17]:
cols = ['sighting_date', 'species', 'number', 'latitude', 'longitude']

In [18]:
JNorth_Egg_Site_df =JNorth_Egg_Site_df.groupby(['sighting_date', 'species', 'latitude', 'longitude']).sum().reset_index()
JNorth_Egg_Site_df.head()

,sighting_date,species,latitude,longitude,number
0,2007-08-12,Monarch Egg Sighted,36.270,-95.855,17
1,2007-09-18,Monarch Egg Sighted,45.227,-93.359,2
2,2012-06-15,Monarch Egg Sighted,33.761,-118.199,1
3,2012-07-12,Monarch Egg Sighted,38.493,-107.073,1
4,2012-07-30,Monarch Egg Sighted,39.424,-77.819,1


In [19]:
JNorth_Egg_Site_df.columns.tolist()

['sighting_date', 'species', 'latitude', 'longitude', 'number']

In [20]:
JNorth_Egg_Site_df.head()

,sighting_date,species,latitude,longitude,number
0,2007-08-12,Monarch Egg Sighted,36.270,-95.855,17
1,2007-09-18,Monarch Egg Sighted,45.227,-93.359,2
2,2012-06-15,Monarch Egg Sighted,33.761,-118.199,1
3,2012-07-12,Monarch Egg Sighted,38.493,-107.073,1
4,2012-07-30,Monarch Egg Sighted,39.424,-77.819,1


In [21]:
JNorth_Egg_Site_df.shape

(8506, 5)

In [22]:
JNorth_Egg_Site_year_df = JNorth_Egg_Site_df
JNorth_Egg_Site_year_df['year'] = pd.DatetimeIndex(JNorth_Egg_Site_df['sighting_date']).year

# 2011

In [23]:
JNorth_Egg_Site_2011_df = JNorth_Egg_Site_year_df[JNorth_Egg_Site_year_df['year']==2011]
JNorth_Egg_Site_2011_df.tail()

,sighting_date,species,latitude,longitude,number,year


In [24]:
JNorth_Egg_Site_2011_df.shape

(0, 6)

In [25]:
JNorth_Egg_Site_2011_df = pd.DataFrame(JNorth_Egg_Site_2011_df)
JNorth_Egg_Site_2011_df.drop(['year'], axis=1, inplace=True)

In [26]:
JNorth_Egg_Site_2011_df['month'] = pd.DatetimeIndex(JNorth_Egg_Site_2011_df['sighting_date']).month

## January

In [27]:
JNorth_Egg_Site_2011_month_df =pd.DataFrame(JNorth_Egg_Site_2011_df)
JNorth_Egg_Site_2011_1_df = JNorth_Egg_Site_2011_month_df[JNorth_Egg_Site_2011_month_df['month']==1]
JNorth_Egg_Site_2011_1_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [28]:
JNorth_Egg_Site_2011_1_df.shape

(0, 6)

In [29]:
JNorth_Egg_Site_2011_1_df = pd.DataFrame(JNorth_Egg_Site_2011_1_df)
JNorth_Egg_Site_2011_1_df.drop(['month'], axis=1, inplace=True)

In [30]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2011_1_df['latitude'] = JNorth_Egg_Site_2011_1_df['latitude'].astype(float)
JNorth_Egg_Site_2011_1_df['longitude'] = JNorth_Egg_Site_2011_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [31]:
geojson = df_to_geojson(JNorth_Egg_Site_2011_1_df, cols)

In [32]:
file = open("JNorth_Egg_Site_2011_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [33]:
JNorth_Egg_Site_2011_month_df =pd.DataFrame(JNorth_Egg_Site_2011_df)
JNorth_Egg_Site_2011_2_df = JNorth_Egg_Site_2011_month_df[JNorth_Egg_Site_2011_month_df['month']==2]
JNorth_Egg_Site_2011_2_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [34]:
JNorth_Egg_Site_2011_2_df.shape

(0, 6)

In [35]:
JNorth_Egg_Site_2011_2_df= pd.DataFrame(JNorth_Egg_Site_2011_2_df)
JNorth_Egg_Site_2011_2_df.drop(['month'], axis=1, inplace=True)

In [36]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2011_2_df['latitude'] = JNorth_Egg_Site_2011_2_df['latitude'].astype(float)
JNorth_Egg_Site_2011_2_df['longitude'] = JNorth_Egg_Site_2011_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [37]:
geojson = df_to_geojson(JNorth_Egg_Site_2011_2_df, cols)

In [38]:
file = open("JNorth_Egg_Site_2011_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [39]:
JNorth_Egg_Site_2011_month_df =pd.DataFrame(JNorth_Egg_Site_2011_df)
JNorth_Egg_Site_2011_3_df = JNorth_Egg_Site_2011_month_df[JNorth_Egg_Site_2011_month_df['month']==3]
JNorth_Egg_Site_2011_3_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [40]:
JNorth_Egg_Site_2011_3_df.shape

(0, 6)

In [41]:
JNorth_Egg_Site_2011_3_df =pd.DataFrame(JNorth_Egg_Site_2011_3_df)
JNorth_Egg_Site_2011_3_df.drop(['month'], axis=1, inplace=True)

In [42]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2011_3_df['latitude'] = JNorth_Egg_Site_2011_3_df['latitude'].astype(float)
JNorth_Egg_Site_2011_3_df['longitude'] = JNorth_Egg_Site_2011_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [43]:
geojson = df_to_geojson(JNorth_Egg_Site_2011_3_df, cols)

In [44]:
file = open("JNorth_Egg_Site_2011_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [45]:
JNorth_Egg_Site_2011_month_df =pd.DataFrame(JNorth_Egg_Site_2011_df)
JNorth_Egg_Site_2011_4_df = JNorth_Egg_Site_2011_month_df[JNorth_Egg_Site_2011_month_df['month']==4]
JNorth_Egg_Site_2011_4_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [46]:
JNorth_Egg_Site_2011_4_df.shape

(0, 6)

In [47]:
JNorth_Egg_Site_2011_4_df = pd.DataFrame(JNorth_Egg_Site_2011_4_df)
JNorth_Egg_Site_2011_4_df.drop(['month'], axis=1, inplace=True)

In [48]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2011_4_df['latitude'] = JNorth_Egg_Site_2011_4_df['latitude'].astype(float)
JNorth_Egg_Site_2011_4_df['longitude'] = JNorth_Egg_Site_2011_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [49]:
geojson = df_to_geojson(JNorth_Egg_Site_2011_4_df, cols)

In [50]:
file = open("JNorth_Egg_Site_2011_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [51]:
JNorth_Egg_Site_2011_month_df =pd.DataFrame(JNorth_Egg_Site_2011_df)
JNorth_Egg_Site_2011_5_df = JNorth_Egg_Site_2011_month_df[JNorth_Egg_Site_2011_month_df['month']==5]
JNorth_Egg_Site_2011_5_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [52]:
JNorth_Egg_Site_2011_5_df.shape

(0, 6)

In [53]:
JNorth_Egg_Site_2011_5_df =pd.DataFrame(JNorth_Egg_Site_2011_5_df)
JNorth_Egg_Site_2011_5_df.drop(['month'], axis=1, inplace=True)

In [54]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2011_5_df['latitude'] = JNorth_Egg_Site_2011_5_df['latitude'].astype(float)
JNorth_Egg_Site_2011_5_df['longitude'] = JNorth_Egg_Site_2011_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [55]:
geojson = df_to_geojson(JNorth_Egg_Site_2011_5_df, cols)

In [56]:
file = open("JNorth_Egg_Site_2011_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [57]:
JNorth_Egg_Site_2011_month_df =pd.DataFrame(JNorth_Egg_Site_2011_df)
JNorth_Egg_Site_2011_6_df = JNorth_Egg_Site_2011_month_df[JNorth_Egg_Site_2011_month_df['month']==6]
JNorth_Egg_Site_2011_6_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [58]:
JNorth_Egg_Site_2011_6_df.shape

(0, 6)

In [59]:
JNorth_Egg_Site_2011_6_df=pd.DataFrame(JNorth_Egg_Site_2011_6_df)
JNorth_Egg_Site_2011_6_df.drop(['month'], axis=1, inplace=True)

In [60]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2011_6_df['latitude'] = JNorth_Egg_Site_2011_6_df['latitude'].astype(float)
JNorth_Egg_Site_2011_6_df['longitude'] = JNorth_Egg_Site_2011_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [61]:
geojson = df_to_geojson(JNorth_Egg_Site_2011_6_df, cols)

In [62]:
file = open("JNorth_Egg_Site_2011_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [63]:
JNorth_Egg_Site_2011_month_df =pd.DataFrame(JNorth_Egg_Site_2011_df)
JNorth_Egg_Site_2011_7_df = JNorth_Egg_Site_2011_month_df[JNorth_Egg_Site_2011_month_df['month']==7]
JNorth_Egg_Site_2011_7_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [64]:
JNorth_Egg_Site_2011_7_df.shape

(0, 6)

In [65]:
JNorth_Egg_Site_2011_7_df = pd.DataFrame(JNorth_Egg_Site_2011_7_df)
JNorth_Egg_Site_2011_7_df.drop(['month'], axis=1, inplace=True)

In [66]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2011_7_df['latitude'] = JNorth_Egg_Site_2011_7_df['latitude'].astype(float)
JNorth_Egg_Site_2011_7_df['longitude'] = JNorth_Egg_Site_2011_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [67]:
geojson = df_to_geojson(JNorth_Egg_Site_2011_7_df, cols)

In [68]:
file = open("JNorth_Egg_Site_2011_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [69]:
JNorth_Egg_Site_2011_month_df =pd.DataFrame(JNorth_Egg_Site_2011_df)
JNorth_Egg_Site_2011_8_df = JNorth_Egg_Site_2011_month_df[JNorth_Egg_Site_2011_month_df['month']==8]
JNorth_Egg_Site_2011_8_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [70]:
JNorth_Egg_Site_2011_8_df.shape

(0, 6)

In [71]:
JNorth_Egg_Site_2011_8_df = pd.DataFrame(JNorth_Egg_Site_2011_8_df)
JNorth_Egg_Site_2011_8_df.drop(['month'], axis=1, inplace=True)

In [72]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2011_8_df['latitude'] = JNorth_Egg_Site_2011_8_df['latitude'].astype(float)
JNorth_Egg_Site_2011_8_df['longitude'] = JNorth_Egg_Site_2011_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [73]:
geojson = df_to_geojson(JNorth_Egg_Site_2011_8_df, cols)

In [74]:
file = open("JNorth_Egg_Site_2011_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [75]:
JNorth_Egg_Site_2011_month_df =pd.DataFrame(JNorth_Egg_Site_2011_df)
JNorth_Egg_Site_2011_9_df = JNorth_Egg_Site_2011_month_df[JNorth_Egg_Site_2011_month_df['month']==9]
JNorth_Egg_Site_2011_9_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [76]:
JNorth_Egg_Site_2011_9_df.shape

(0, 6)

In [77]:
JNorth_Egg_Site_2011_9_df = pd.DataFrame(JNorth_Egg_Site_2011_9_df)
JNorth_Egg_Site_2011_9_df.drop(['month'], axis=1, inplace=True)

In [78]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2011_9_df['latitude'] = JNorth_Egg_Site_2011_9_df['latitude'].astype(float)
JNorth_Egg_Site_2011_9_df['longitude'] = JNorth_Egg_Site_2011_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [79]:
geojson = df_to_geojson(JNorth_Egg_Site_2011_9_df, cols)

In [80]:
file = open("JNorth_Egg_Site_2011_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [81]:
JNorth_Egg_Site_2011_month_df =pd.DataFrame(JNorth_Egg_Site_2011_df)
JNorth_Egg_Site_2011_10_df = JNorth_Egg_Site_2011_month_df[JNorth_Egg_Site_2011_month_df['month']==10]
JNorth_Egg_Site_2011_10_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [82]:
JNorth_Egg_Site_2011_10_df.shape

(0, 6)

In [83]:
JNorth_Egg_Site_2011_10_df = pd.DataFrame(JNorth_Egg_Site_2011_10_df)
JNorth_Egg_Site_2011_10_df.drop(['month'], axis=1, inplace=True)

In [84]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2011_10_df['latitude'] = JNorth_Egg_Site_2011_10_df['latitude'].astype(float)
JNorth_Egg_Site_2011_10_df['longitude'] = JNorth_Egg_Site_2011_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [85]:
geojson = df_to_geojson(JNorth_Egg_Site_2011_10_df, cols)

In [86]:
file = open("JNorth_Egg_Site_2011_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [87]:
JNorth_Egg_Site_2011_month_df =pd.DataFrame(JNorth_Egg_Site_2011_df)
JNorth_Egg_Site_2011_11_df = JNorth_Egg_Site_2011_month_df[JNorth_Egg_Site_2011_month_df['month']==11]
JNorth_Egg_Site_2011_11_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [88]:
JNorth_Egg_Site_2011_11_df.shape

(0, 6)

In [89]:
JNorth_Egg_Site_2011_11_df = pd.DataFrame(JNorth_Egg_Site_2011_11_df)
JNorth_Egg_Site_2011_11_df.drop(['month'], axis=1, inplace=True)

In [90]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2011_11_df['latitude'] = JNorth_Egg_Site_2011_11_df['latitude'].astype(float)
JNorth_Egg_Site_2011_11_df['longitude'] = JNorth_Egg_Site_2011_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [91]:
geojson = df_to_geojson(JNorth_Egg_Site_2011_11_df, cols)

In [92]:
file = open("JNorth_Egg_Site_2011_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [93]:
JNorth_Egg_Site_2011_month_df =pd.DataFrame(JNorth_Egg_Site_2011_df)
JNorth_Egg_Site_2011_12_df = JNorth_Egg_Site_2011_month_df[JNorth_Egg_Site_2011_month_df['month']==12]
JNorth_Egg_Site_2011_12_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [94]:
JNorth_Egg_Site_2011_12_df.shape

(0, 6)

In [95]:
JNorth_Egg_Site_2011_12_df = pd.DataFrame(JNorth_Egg_Site_2011_12_df)
JNorth_Egg_Site_2011_12_df.drop(['month'], axis=1, inplace=True)

In [96]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2011_12_df['latitude'] = JNorth_Egg_Site_2011_12_df['latitude'].astype(float)
JNorth_Egg_Site_2011_12_df['longitude'] = JNorth_Egg_Site_2011_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [97]:
geojson = df_to_geojson(JNorth_Egg_Site_2011_12_df, cols)

In [98]:
file = open("JNorth_Egg_Site_2011_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# 2012

In [99]:
JNorth_Egg_Site_2012_df = JNorth_Egg_Site_year_df[JNorth_Egg_Site_year_df['year']==2012]
JNorth_Egg_Site_2012_df.tail()

,sighting_date,species,latitude,longitude,number,year
186,2012-11-08,Monarch Egg Sighted,33.477,-93.918,1,2012
187,2012-11-22,Monarch Egg Sighted,34.238,-118.549,1,2012
188,2012-11-24,Monarch Egg Sighted,27.770,-82.637,1,2012
189,2012-11-28,Monarch Egg Sighted,32.216,-110.971,1,2012
190,2012-12-11,Monarch Egg Sighted,32.845,-79.811,100,2012


In [100]:
JNorth_Egg_Site_2012_df.shape

(189, 6)

In [101]:
JNorth_Egg_Site_2012_df = pd.DataFrame(JNorth_Egg_Site_2012_df)
JNorth_Egg_Site_2012_df.drop(['year'], axis=1, inplace=True)

In [102]:
JNorth_Egg_Site_2012_df.tail

<bound method NDFrame.tail of     sighting_date              species  latitude  longitude  number
2      2012-06-15  Monarch Egg Sighted    33.761   -118.199       1
3      2012-07-12  Monarch Egg Sighted    38.493   -107.073       1
4      2012-07-30  Monarch Egg Sighted    39.424    -77.819       1
5      2012-08-02  Monarch Egg Sighted    39.091    -76.624       4
6      2012-08-02  Monarch Egg Sighted    42.647    -74.575       1
..            ...                  ...       ...        ...     ...
186    2012-11-08  Monarch Egg Sighted    33.477    -93.918       1
187    2012-11-22  Monarch Egg Sighted    34.238   -118.549       1
188    2012-11-24  Monarch Egg Sighted    27.770    -82.637       1
189    2012-11-28  Monarch Egg Sighted    32.216   -110.971       1
190    2012-12-11  Monarch Egg Sighted    32.845    -79.811     100

[189 rows x 5 columns]>

In [103]:
JNorth_Egg_Site_2012_df['month'] = pd.DatetimeIndex(JNorth_Egg_Site_2012_df['sighting_date']).month

## January

In [104]:
JNorth_Egg_Site_2012_month_df =pd.DataFrame(JNorth_Egg_Site_2012_df)
JNorth_Egg_Site_2012_1_df = JNorth_Egg_Site_2012_month_df[JNorth_Egg_Site_2012_month_df['month']==1]
JNorth_Egg_Site_2012_1_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [105]:
JNorth_Egg_Site_2012_1_df.shape

(0, 6)

In [106]:
JNorth_Egg_Site_2012_1_df = pd.DataFrame(JNorth_Egg_Site_2012_1_df)
JNorth_Egg_Site_2012_1_df.drop(['month'], axis=1, inplace=True)

In [107]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2012_1_df['latitude'] = JNorth_Egg_Site_2012_1_df['latitude'].astype(float)
JNorth_Egg_Site_2012_1_df['longitude'] = JNorth_Egg_Site_2012_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [108]:
geojson = df_to_geojson(JNorth_Egg_Site_2012_1_df, cols)

In [109]:
file = open("JNorth_Egg_Site_2012_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [110]:
JNorth_Egg_Site_2012_month_df =pd.DataFrame(JNorth_Egg_Site_2012_df)
JNorth_Egg_Site_2012_2_df = JNorth_Egg_Site_2012_month_df[JNorth_Egg_Site_2012_month_df['month']==2]
JNorth_Egg_Site_2012_2_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [111]:
JNorth_Egg_Site_2012_2_df.shape

(0, 6)

In [112]:
JNorth_Egg_Site_2012_2_df = pd.DataFrame(JNorth_Egg_Site_2012_2_df)
JNorth_Egg_Site_2012_2_df.drop(['month'], axis=1, inplace=True)

In [113]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2012_2_df['latitude'] = JNorth_Egg_Site_2012_2_df['latitude'].astype(float)
JNorth_Egg_Site_2012_2_df['longitude'] = JNorth_Egg_Site_2012_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [114]:
geojson = df_to_geojson(JNorth_Egg_Site_2012_2_df, cols)

In [115]:
file = open("JNorth_Egg_Site_2012_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [116]:
JNorth_Egg_Site_2012_month_df =pd.DataFrame(JNorth_Egg_Site_2012_df)
JNorth_Egg_Site_2012_3_df = JNorth_Egg_Site_2012_month_df[JNorth_Egg_Site_2012_month_df['month']==3]
JNorth_Egg_Site_2012_3_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [117]:
JNorth_Egg_Site_2012_3_df.shape

(0, 6)

In [118]:
JNorth_Egg_Site_2012_3_df = pd.DataFrame(JNorth_Egg_Site_2012_3_df)
JNorth_Egg_Site_2012_3_df.drop(['month'], axis=1, inplace=True)

In [119]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2012_3_df['latitude'] = JNorth_Egg_Site_2012_3_df['latitude'].astype(float)
JNorth_Egg_Site_2012_3_df['longitude'] = JNorth_Egg_Site_2012_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [120]:
geojson = df_to_geojson(JNorth_Egg_Site_2012_3_df, cols)

In [121]:
file = open("JNorth_Egg_Site_2012_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [122]:
JNorth_Egg_Site_2012_month_df =pd.DataFrame(JNorth_Egg_Site_2012_df)
JNorth_Egg_Site_2012_4_df = JNorth_Egg_Site_2012_month_df[JNorth_Egg_Site_2012_month_df['month']==4]
JNorth_Egg_Site_2012_4_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [123]:
JNorth_Egg_Site_2012_4_df.shape

(0, 6)

In [124]:
JNorth_Egg_Site_2012_4_df = pd.DataFrame(JNorth_Egg_Site_2012_4_df)
JNorth_Egg_Site_2012_4_df.drop(['month'], axis=1, inplace=True)

In [125]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2012_4_df['latitude'] = JNorth_Egg_Site_2012_4_df['latitude'].astype(float)
JNorth_Egg_Site_2012_4_df['longitude'] = JNorth_Egg_Site_2012_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [126]:
geojson = df_to_geojson(JNorth_Egg_Site_2012_4_df, cols)

In [127]:
file = open("JNorth_Egg_Site_2012_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [128]:
JNorth_Egg_Site_2012_month_df =pd.DataFrame(JNorth_Egg_Site_2012_df)
JNorth_Egg_Site_2012_5_df = JNorth_Egg_Site_2012_month_df[JNorth_Egg_Site_2012_month_df['month']==5]
JNorth_Egg_Site_2012_5_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [129]:
JNorth_Egg_Site_2012_5_df.shape

(0, 6)

In [130]:
JNorth_Egg_Site_2012_5_df = pd.DataFrame(JNorth_Egg_Site_2012_5_df)
JNorth_Egg_Site_2012_5_df.drop(['month'], axis=1, inplace=True)

In [131]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2012_5_df['latitude'] = JNorth_Egg_Site_2012_5_df['latitude'].astype(float)
JNorth_Egg_Site_2012_5_df['longitude'] = JNorth_Egg_Site_2012_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [132]:
geojson = df_to_geojson(JNorth_Egg_Site_2012_5_df, cols)

In [133]:
file = open("JNorth_Egg_Site_2012_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [134]:
JNorth_Egg_Site_2012_month_df =pd.DataFrame(JNorth_Egg_Site_2012_df)
JNorth_Egg_Site_2012_6_df = JNorth_Egg_Site_2012_month_df[JNorth_Egg_Site_2012_month_df['month']==6]
JNorth_Egg_Site_2012_6_df.tail()

,sighting_date,species,latitude,longitude,number,month
2,2012-06-15,Monarch Egg Sighted,33.761,-118.199,1,6


In [135]:
JNorth_Egg_Site_2012_6_df.shape

(1, 6)

In [136]:
JNorth_Egg_Site_2012_6_df = pd.DataFrame(JNorth_Egg_Site_2012_6_df)
JNorth_Egg_Site_2012_6_df.drop(['month'], axis=1, inplace=True)

In [137]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2012_6_df['latitude'] = JNorth_Egg_Site_2012_6_df['latitude'].astype(float)
JNorth_Egg_Site_2012_6_df['longitude'] = JNorth_Egg_Site_2012_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [138]:
geojson = df_to_geojson(JNorth_Egg_Site_2012_6_df, cols)

In [139]:
file = open("JNorth_Egg_Site_2012_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [140]:
JNorth_Egg_Site_2012_month_df =pd.DataFrame(JNorth_Egg_Site_2012_df)
JNorth_Egg_Site_2012_7_df = JNorth_Egg_Site_2012_month_df[JNorth_Egg_Site_2012_month_df['month']==7]
JNorth_Egg_Site_2012_7_df.tail()

,sighting_date,species,latitude,longitude,number,month
3,2012-07-12,Monarch Egg Sighted,38.493,-107.073,1,7
4,2012-07-30,Monarch Egg Sighted,39.424,-77.819,1,7


In [141]:
JNorth_Egg_Site_2012_7_df.shape

(2, 6)

In [142]:
JNorth_Egg_Site_2012_7_df = pd.DataFrame(JNorth_Egg_Site_2012_7_df)
JNorth_Egg_Site_2012_7_df.drop(['month'], axis=1, inplace=True)

In [143]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2012_7_df['latitude'] = JNorth_Egg_Site_2012_7_df['latitude'].astype(float)
JNorth_Egg_Site_2012_7_df['longitude'] = JNorth_Egg_Site_2012_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [144]:
geojson = df_to_geojson(JNorth_Egg_Site_2012_7_df, cols)

In [145]:
file = open("JNorth_Egg_Site_2012_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [146]:
JNorth_Egg_Site_2012_month_df =pd.DataFrame(JNorth_Egg_Site_2012_df)
JNorth_Egg_Site_2012_8_df = JNorth_Egg_Site_2012_month_df[JNorth_Egg_Site_2012_month_df['month']==8]
JNorth_Egg_Site_2012_8_df.tail()

,sighting_date,species,latitude,longitude,number,month
98,2012-08-30,Monarch Egg Sighted,42.145,-72.607,5,8
99,2012-08-30,Monarch Egg Sighted,42.375,-85.708,1,8
100,2012-08-31,Monarch Egg Sighted,30.449,-95.489,1,8
101,2012-08-31,Monarch Egg Sighted,32.804,-79.755,2,8
102,2012-08-31,Monarch Egg Sighted,39.743,-76.046,1,8


In [147]:
JNorth_Egg_Site_2012_8_df.shape

(98, 6)

In [148]:
JNorth_Egg_Site_2012_8_df = pd.DataFrame(JNorth_Egg_Site_2012_8_df)
JNorth_Egg_Site_2012_8_df.drop(['month'], axis=1, inplace=True)

In [149]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2012_8_df['latitude'] = JNorth_Egg_Site_2012_8_df['latitude'].astype(float)
JNorth_Egg_Site_2012_8_df['longitude'] = JNorth_Egg_Site_2012_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [150]:
geojson = df_to_geojson(JNorth_Egg_Site_2012_8_df, cols)

In [151]:
file = open("JNorth_Egg_Site_2012_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [152]:
JNorth_Egg_Site_2012_month_df =pd.DataFrame(JNorth_Egg_Site_2012_df)
JNorth_Egg_Site_2012_9_df = JNorth_Egg_Site_2012_month_df[JNorth_Egg_Site_2012_month_df['month']==9]
JNorth_Egg_Site_2012_9_df.tail()

,sighting_date,species,latitude,longitude,number,month
161,2012-09-26,Monarch Egg Sighted,31.536,-92.381,1,9
162,2012-09-27,Monarch Egg Sighted,25.778,-80.198,1,9
163,2012-09-27,Monarch Egg Sighted,32.758,-97.328,1,9
164,2012-09-28,Monarch Egg Sighted,40.594,-74.624,1,9
165,2012-09-29,Monarch Egg Sighted,33.680,-117.907,2,9


In [153]:
JNorth_Egg_Site_2012_9_df.shape

(63, 6)

In [154]:
JNorth_Egg_Site_2012_9_df = pd.DataFrame(JNorth_Egg_Site_2012_9_df)
JNorth_Egg_Site_2012_9_df.drop(['month'], axis=1, inplace=True)

In [155]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2012_9_df['latitude'] = JNorth_Egg_Site_2012_9_df['latitude'].astype(float)
JNorth_Egg_Site_2012_9_df['longitude'] = JNorth_Egg_Site_2012_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [156]:
geojson = df_to_geojson(JNorth_Egg_Site_2012_9_df, cols)

In [157]:
file = open("JNorth_Egg_Site_2012_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [158]:
JNorth_Egg_Site_2012_month_df =pd.DataFrame(JNorth_Egg_Site_2012_df)
JNorth_Egg_Site_2012_10_df = JNorth_Egg_Site_2012_month_df[JNorth_Egg_Site_2012_month_df['month']==10]
JNorth_Egg_Site_2012_10_df.tail()

,sighting_date,species,latitude,longitude,number,month
180,2012-10-20,Monarch Egg Sighted,30.930,-97.557,1,10
181,2012-10-25,Monarch Egg Sighted,28.078,-80.627,1,10
182,2012-10-27,Monarch Egg Sighted,29.757,-95.365,6,10
183,2012-10-27,Monarch Egg Sighted,29.984,-90.166,50,10
184,2012-10-30,Monarch Egg Sighted,29.501,-95.192,1,10


In [159]:
JNorth_Egg_Site_2012_10_df.shape

(19, 6)

In [160]:
JNorth_Egg_Site_2012_10_df = pd.DataFrame(JNorth_Egg_Site_2012_10_df)
JNorth_Egg_Site_2012_10_df.drop(['month'], axis=1, inplace=True)

In [161]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2012_10_df['latitude'] = JNorth_Egg_Site_2012_10_df['latitude'].astype(float)
JNorth_Egg_Site_2012_10_df['longitude'] = JNorth_Egg_Site_2012_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [162]:
geojson = df_to_geojson(JNorth_Egg_Site_2012_10_df, cols)

In [163]:
file = open("JNorth_Egg_Site_2012_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [164]:
JNorth_Egg_Site_2012_month_df =pd.DataFrame(JNorth_Egg_Site_2012_df)
JNorth_Egg_Site_2012_11_df = JNorth_Egg_Site_2012_month_df[JNorth_Egg_Site_2012_month_df['month']==11]
JNorth_Egg_Site_2012_11_df.tail()

,sighting_date,species,latitude,longitude,number,month
185,2012-11-07,Monarch Egg Sighted,29.895,-81.314,1,11
186,2012-11-08,Monarch Egg Sighted,33.477,-93.918,1,11
187,2012-11-22,Monarch Egg Sighted,34.238,-118.549,1,11
188,2012-11-24,Monarch Egg Sighted,27.770,-82.637,1,11
189,2012-11-28,Monarch Egg Sighted,32.216,-110.971,1,11


In [165]:
JNorth_Egg_Site_2012_11_df.shape

(5, 6)

In [166]:
JNorth_Egg_Site_2012_11_df = pd.DataFrame(JNorth_Egg_Site_2012_11_df)
JNorth_Egg_Site_2012_11_df.drop(['month'], axis=1, inplace=True)

In [167]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2012_11_df['latitude'] = JNorth_Egg_Site_2012_11_df['latitude'].astype(float)
JNorth_Egg_Site_2012_11_df['longitude'] = JNorth_Egg_Site_2012_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [168]:
geojson = df_to_geojson(JNorth_Egg_Site_2012_11_df, cols)

In [169]:
file = open("JNorth_Egg_Site_2012_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [170]:
JNorth_Egg_Site_2012_month_df =pd.DataFrame(JNorth_Egg_Site_2012_df)
JNorth_Egg_Site_2012_12_df = JNorth_Egg_Site_2012_month_df[JNorth_Egg_Site_2012_month_df['month']==12]
JNorth_Egg_Site_2012_12_df.tail()

,sighting_date,species,latitude,longitude,number,month
190,2012-12-11,Monarch Egg Sighted,32.845,-79.811,100,12


In [171]:
JNorth_Egg_Site_2012_12_df.shape

(1, 6)

In [172]:
JNorth_Egg_Site_2012_12_df = pd.DataFrame(JNorth_Egg_Site_2012_12_df)
JNorth_Egg_Site_2012_12_df.drop(['month'], axis=1, inplace=True)

In [173]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2012_12_df['latitude'] = JNorth_Egg_Site_2012_12_df['latitude'].astype(float)
JNorth_Egg_Site_2012_12_df['longitude'] = JNorth_Egg_Site_2012_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [174]:
geojson = df_to_geojson(JNorth_Egg_Site_2012_12_df, cols)

In [175]:
file = open("JNorth_Egg_Site_2012_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# 2013

In [176]:
JNorth_Egg_Site_2013_df = JNorth_Egg_Site_year_df[JNorth_Egg_Site_year_df['year']==2013]
JNorth_Egg_Site_2013_df.tail()

,sighting_date,species,latitude,longitude,number,year
425,2013-12-01,Monarch Egg Sighted,33.761,-118.199,12,2013
426,2013-12-11,Monarch Egg Sighted,29.280,-81.080,1,2013
427,2013-12-12,Monarch Egg Sighted,33.761,-118.199,1,2013
428,2013-12-23,Monarch Egg Sighted,42.955,-87.944,1,2013
429,2013-12-31,Monarch Egg Sighted,30.420,-87.224,1,2013


In [177]:
JNorth_Egg_Site_2013_df.shape

(239, 6)

In [178]:
JNorth_Egg_Site_2013_df = pd.DataFrame(JNorth_Egg_Site_2013_df)
JNorth_Egg_Site_2013_df.drop(['year'], axis=1, inplace=True)

In [179]:
JNorth_Egg_Site_2013_df.tail

<bound method NDFrame.tail of     sighting_date              species  latitude  longitude  number
191    2013-07-04  Monarch Egg Sighted    40.819    -74.158       1
192    2013-07-04  Monarch Egg Sighted    44.986    -93.270       1
193    2013-07-19  Monarch Egg Sighted    42.118    -88.096       1
194    2013-07-21  Monarch Egg Sighted    42.682    -70.849       4
195    2013-07-21  Monarch Egg Sighted    43.950    -79.833      12
..            ...                  ...       ...        ...     ...
425    2013-12-01  Monarch Egg Sighted    33.761   -118.199      12
426    2013-12-11  Monarch Egg Sighted    29.280    -81.080       1
427    2013-12-12  Monarch Egg Sighted    33.761   -118.199       1
428    2013-12-23  Monarch Egg Sighted    42.955    -87.944       1
429    2013-12-31  Monarch Egg Sighted    30.420    -87.224       1

[239 rows x 5 columns]>

In [180]:
JNorth_Egg_Site_2013_df['month'] = pd.DatetimeIndex(JNorth_Egg_Site_2013_df['sighting_date']).month

## January

In [181]:
JNorth_Egg_Site_2013_month_df =pd.DataFrame(JNorth_Egg_Site_2013_df)
JNorth_Egg_Site_2013_1_df = JNorth_Egg_Site_2013_month_df[JNorth_Egg_Site_2013_month_df['month']==1]
JNorth_Egg_Site_2013_1_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [182]:
JNorth_Egg_Site_2013_1_df.shape

(0, 6)

In [183]:
JNorth_Egg_Site_2013_1_df = pd.DataFrame(JNorth_Egg_Site_2013_1_df)
JNorth_Egg_Site_2013_1_df.drop(['month'], axis=1, inplace=True)

In [184]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2013_1_df['latitude'] = JNorth_Egg_Site_2013_1_df['latitude'].astype(float)
JNorth_Egg_Site_2013_1_df['longitude'] = JNorth_Egg_Site_2013_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [185]:
geojson = df_to_geojson(JNorth_Egg_Site_2013_1_df, cols)

In [186]:
file = open("JNorth_Egg_Site_2013_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [187]:
JNorth_Egg_Site_2013_month_df =pd.DataFrame(JNorth_Egg_Site_2013_df)
JNorth_Egg_Site_2013_2_df = JNorth_Egg_Site_2013_month_df[JNorth_Egg_Site_2013_month_df['month']==2]
JNorth_Egg_Site_2013_2_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [188]:
JNorth_Egg_Site_2013_2_df.shape

(0, 6)

In [189]:
JNorth_Egg_Site_2013_2_df = pd.DataFrame(JNorth_Egg_Site_2013_2_df)
JNorth_Egg_Site_2013_2_df.drop(['month'], axis=1, inplace=True)

In [190]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2013_2_df['latitude'] = JNorth_Egg_Site_2013_2_df['latitude'].astype(float)
JNorth_Egg_Site_2013_2_df['longitude'] = JNorth_Egg_Site_2013_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [191]:
geojson = df_to_geojson(JNorth_Egg_Site_2013_2_df, cols)

In [192]:
file = open("JNorth_Egg_Site_2013_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [193]:
JNorth_Egg_Site_2013_month_df =pd.DataFrame(JNorth_Egg_Site_2013_df)
JNorth_Egg_Site_2013_3_df = JNorth_Egg_Site_2013_month_df[JNorth_Egg_Site_2013_month_df['month']==3]
JNorth_Egg_Site_2013_3_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [194]:
JNorth_Egg_Site_2013_3_df.shape

(0, 6)

In [195]:
JNorth_Egg_Site_2013_3_df = pd.DataFrame(JNorth_Egg_Site_2013_3_df)
JNorth_Egg_Site_2013_3_df.drop(['month'], axis=1, inplace=True)

In [196]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2013_3_df['latitude'] = JNorth_Egg_Site_2013_3_df['latitude'].astype(float)
JNorth_Egg_Site_2013_3_df['longitude'] = JNorth_Egg_Site_2013_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [197]:
geojson = df_to_geojson(JNorth_Egg_Site_2013_3_df, cols)

In [198]:
file = open("JNorth_Egg_Site_2013_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [199]:
JNorth_Egg_Site_2013_month_df =pd.DataFrame(JNorth_Egg_Site_2013_df)
JNorth_Egg_Site_2013_4_df = JNorth_Egg_Site_2013_month_df[JNorth_Egg_Site_2013_month_df['month']==4]
JNorth_Egg_Site_2013_4_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [200]:
JNorth_Egg_Site_2013_4_df.shape

(0, 6)

In [201]:
JNorth_Egg_Site_2013_4_df =pd.DataFrame(JNorth_Egg_Site_2013_4_df)
JNorth_Egg_Site_2013_4_df.drop(['month'], axis=1, inplace=True)

In [202]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2013_4_df['latitude'] = JNorth_Egg_Site_2013_4_df['latitude'].astype(float)
JNorth_Egg_Site_2013_4_df['longitude'] = JNorth_Egg_Site_2013_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [203]:
geojson = df_to_geojson(JNorth_Egg_Site_2013_4_df, cols)

In [204]:
file = open("JNorth_Egg_Site_2013_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [205]:
JNorth_Egg_Site_2013_month_df =pd.DataFrame(JNorth_Egg_Site_2013_df)
JNorth_Egg_Site_2013_5_df = JNorth_Egg_Site_2013_month_df[JNorth_Egg_Site_2013_month_df['month']==5]
JNorth_Egg_Site_2013_5_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [206]:
JNorth_Egg_Site_2013_5_df.shape

(0, 6)

In [207]:
JNorth_Egg_Site_2013_5_df =pd.DataFrame(JNorth_Egg_Site_2013_5_df)
JNorth_Egg_Site_2013_5_df.drop(['month'], axis=1, inplace=True)

In [208]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2013_5_df['latitude'] = JNorth_Egg_Site_2013_5_df['latitude'].astype(float)
JNorth_Egg_Site_2013_5_df['longitude'] = JNorth_Egg_Site_2013_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [209]:
geojson = df_to_geojson(JNorth_Egg_Site_2013_5_df, cols)

In [210]:
file = open("JNorth_Egg_Site_2013_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [211]:
JNorth_Egg_Site_2013_month_df =pd.DataFrame(JNorth_Egg_Site_2013_df)
JNorth_Egg_Site_2013_6_df = JNorth_Egg_Site_2013_month_df[JNorth_Egg_Site_2013_month_df['month']==6]
JNorth_Egg_Site_2013_6_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [212]:
JNorth_Egg_Site_2013_6_df.shape

(0, 6)

In [213]:
JNorth_Egg_Site_2013_6_df = pd.DataFrame(JNorth_Egg_Site_2013_6_df)
JNorth_Egg_Site_2013_6_df.drop(['month'], axis=1, inplace=True)

In [214]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2013_6_df['latitude'] = JNorth_Egg_Site_2013_6_df['latitude'].astype(float)
JNorth_Egg_Site_2013_6_df['longitude'] = JNorth_Egg_Site_2013_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [215]:
geojson = df_to_geojson(JNorth_Egg_Site_2013_6_df, cols)

In [216]:
file = open("JNorth_Egg_Site_2013_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [217]:
JNorth_Egg_Site_2013_month_df =pd.DataFrame(JNorth_Egg_Site_2013_df)
JNorth_Egg_Site_2013_7_df = JNorth_Egg_Site_2013_month_df[JNorth_Egg_Site_2013_month_df['month']==7]
JNorth_Egg_Site_2013_7_df.tail()

,sighting_date,species,latitude,longitude,number,month
194,2013-07-21,Monarch Egg Sighted,42.682,-70.849,4,7
195,2013-07-21,Monarch Egg Sighted,43.950,-79.833,12,7
196,2013-07-28,Monarch Egg Sighted,35.988,-94.089,3,7
197,2013-07-29,Monarch Egg Sighted,43.700,-79.417,3,7
198,2013-07-30,Monarch Egg Sighted,40.025,-85.440,1,7


In [218]:
JNorth_Egg_Site_2013_7_df.shape

(8, 6)

In [219]:
JNorth_Egg_Site_2013_7_df = pd.DataFrame(JNorth_Egg_Site_2013_7_df)
JNorth_Egg_Site_2013_7_df.drop(['month'], axis=1, inplace=True)

In [220]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2013_7_df['latitude'] = JNorth_Egg_Site_2013_7_df['latitude'].astype(float)
JNorth_Egg_Site_2013_7_df['longitude'] = JNorth_Egg_Site_2013_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [221]:
geojson = df_to_geojson(JNorth_Egg_Site_2013_7_df, cols)

In [222]:
file = open("JNorth_Egg_Site_2013_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [223]:
JNorth_Egg_Site_2013_month_df =pd.DataFrame(JNorth_Egg_Site_2013_df)
JNorth_Egg_Site_2013_8_df = JNorth_Egg_Site_2013_month_df[JNorth_Egg_Site_2013_month_df['month']==8]
JNorth_Egg_Site_2013_8_df.tail()

,sighting_date,species,latitude,longitude,number,month
336,2013-08-30,Monarch Egg Sighted,38.393,-85.267,1,8
337,2013-08-30,Monarch Egg Sighted,39.908,-85.923,1,8
338,2013-08-30,Monarch Egg Sighted,40.963,-72.185,1,8
339,2013-08-30,Monarch Egg Sighted,41.139,-100.777,10,8
340,2013-08-31,Monarch Egg Sighted,34.043,-84.707,1,8


In [224]:
JNorth_Egg_Site_2013_8_df.shape

(142, 6)

In [225]:
JNorth_Egg_Site_2013_8_df = pd.DataFrame(JNorth_Egg_Site_2013_8_df)
JNorth_Egg_Site_2013_8_df.drop(['month'], axis=1, inplace=True)

In [226]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2013_8_df['latitude'] = JNorth_Egg_Site_2013_8_df['latitude'].astype(float)
JNorth_Egg_Site_2013_8_df['longitude'] = JNorth_Egg_Site_2013_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [227]:
geojson = df_to_geojson(JNorth_Egg_Site_2013_8_df, cols)

In [228]:
file = open("JNorth_Egg_Site_2013_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [229]:
JNorth_Egg_Site_2013_month_df =pd.DataFrame(JNorth_Egg_Site_2013_df)
JNorth_Egg_Site_2013_9_df = JNorth_Egg_Site_2013_month_df[JNorth_Egg_Site_2013_month_df['month']==9]
JNorth_Egg_Site_2013_9_df.tail()

,sighting_date,species,latitude,longitude,number,month
402,2013-09-28,Monarch Egg Sighted,25.670,-100.320,1,9
403,2013-09-28,Monarch Egg Sighted,32.030,-84.355,1,9
404,2013-09-28,Monarch Egg Sighted,32.131,-97.855,1,9
405,2013-09-29,Monarch Egg Sighted,32.131,-97.855,1,9
406,2013-09-30,Monarch Egg Sighted,32.131,-97.855,1,9


In [230]:
JNorth_Egg_Site_2013_9_df.shape

(66, 6)

In [231]:
JNorth_Egg_Site_2013_9_df = pd.DataFrame(JNorth_Egg_Site_2013_9_df)
JNorth_Egg_Site_2013_9_df.drop(['month'], axis=1, inplace=True)

In [232]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2013_9_df['latitude'] = JNorth_Egg_Site_2013_9_df['latitude'].astype(float)
JNorth_Egg_Site_2013_9_df['longitude'] = JNorth_Egg_Site_2013_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [233]:
geojson = df_to_geojson(JNorth_Egg_Site_2013_9_df, cols)

In [234]:
file = open("JNorth_Egg_Site_2013_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [235]:
JNorth_Egg_Site_2013_month_df =pd.DataFrame(JNorth_Egg_Site_2013_df)
JNorth_Egg_Site_2013_10_df = JNorth_Egg_Site_2013_month_df[JNorth_Egg_Site_2013_month_df['month']==10]
JNorth_Egg_Site_2013_10_df.tail()

,sighting_date,species,latitude,longitude,number,month
415,2013-10-12,Monarch Egg Sighted,32.030,-84.355,1,10
416,2013-10-17,Monarch Egg Sighted,30.613,-96.320,1,10
417,2013-10-22,Monarch Egg Sighted,30.061,-95.384,1,10
418,2013-10-24,Monarch Egg Sighted,28.513,-100.527,1,10
419,2013-10-30,Monarch Egg Sighted,28.827,-82.256,1,10


In [236]:
JNorth_Egg_Site_2013_10_df.shape

(13, 6)

In [237]:
JNorth_Egg_Site_2013_10_df = pd.DataFrame(JNorth_Egg_Site_2013_10_df)
JNorth_Egg_Site_2013_10_df.drop(['month'], axis=1, inplace=True)

In [238]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2013_10_df['latitude'] = JNorth_Egg_Site_2013_10_df['latitude'].astype(float)
JNorth_Egg_Site_2013_10_df['longitude'] = JNorth_Egg_Site_2013_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [239]:
geojson = df_to_geojson(JNorth_Egg_Site_2013_10_df, cols)

In [240]:
file = open("JNorth_Egg_Site_2013_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [241]:
JNorth_Egg_Site_2013_month_df =pd.DataFrame(JNorth_Egg_Site_2013_df)
JNorth_Egg_Site_2013_11_df = JNorth_Egg_Site_2013_month_df[JNorth_Egg_Site_2013_month_df['month']==11]
JNorth_Egg_Site_2013_11_df.tail()

,sighting_date,species,latitude,longitude,number,month
420,2013-11-02,Monarch Egg Sighted,29.501,-95.192,1,11
421,2013-11-15,Monarch Egg Sighted,33.816,-117.823,1,11
422,2013-11-16,Monarch Egg Sighted,28.827,-82.256,1,11
423,2013-11-18,Monarch Egg Sighted,29.757,-95.365,2,11
424,2013-11-30,Monarch Egg Sighted,34.141,-118.978,1,11


In [242]:
JNorth_Egg_Site_2013_11_df.shape

(5, 6)

In [243]:
JNorth_Egg_Site_2013_11_df = pd.DataFrame(JNorth_Egg_Site_2013_11_df)
JNorth_Egg_Site_2013_11_df.drop(['month'], axis=1, inplace=True)

In [244]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2013_11_df['latitude'] = JNorth_Egg_Site_2013_11_df['latitude'].astype(float)
JNorth_Egg_Site_2013_11_df['longitude'] = JNorth_Egg_Site_2013_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [245]:
geojson = df_to_geojson(JNorth_Egg_Site_2013_11_df, cols)

In [246]:
file = open("JNorth_Egg_Site_2013_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [247]:
JNorth_Egg_Site_2013_month_df =pd.DataFrame(JNorth_Egg_Site_2013_df)
JNorth_Egg_Site_2013_12_df = JNorth_Egg_Site_2013_month_df[JNorth_Egg_Site_2013_month_df['month']==12]
JNorth_Egg_Site_2013_12_df.tail()

,sighting_date,species,latitude,longitude,number,month
425,2013-12-01,Monarch Egg Sighted,33.761,-118.199,12,12
426,2013-12-11,Monarch Egg Sighted,29.280,-81.080,1,12
427,2013-12-12,Monarch Egg Sighted,33.761,-118.199,1,12
428,2013-12-23,Monarch Egg Sighted,42.955,-87.944,1,12
429,2013-12-31,Monarch Egg Sighted,30.420,-87.224,1,12


In [248]:
JNorth_Egg_Site_2013_12_df.shape

(5, 6)

In [249]:
JNorth_Egg_Site_2013_12_df = pd.DataFrame(JNorth_Egg_Site_2013_12_df)
JNorth_Egg_Site_2013_12_df.drop(['month'], axis=1, inplace=True)

In [250]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2013_12_df['latitude'] = JNorth_Egg_Site_2013_12_df['latitude'].astype(float)
JNorth_Egg_Site_2013_12_df['longitude'] = JNorth_Egg_Site_2013_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [251]:
geojson = df_to_geojson(JNorth_Egg_Site_2013_12_df, cols)

In [252]:
file = open("JNorth_Egg_Site_2013_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# 2014

In [253]:
JNorth_Egg_Site_2014_df = JNorth_Egg_Site_year_df[JNorth_Egg_Site_year_df['year']==2014]
JNorth_Egg_Site_2014_df.tail()

,sighting_date,species,latitude,longitude,number,year
935,2014-11-24,Monarch Egg Sighted,30.047,-95.214,3,2014
936,2014-12-06,Monarch Egg Sighted,29.757,-95.365,1,2014
937,2014-12-09,Monarch Egg Sighted,30.532,-88.171,1,2014
938,2014-12-14,Monarch Egg Sighted,27.797,-97.399,1,2014
939,2014-12-28,Monarch Egg Sighted,33.834,-118.313,1,2014


In [254]:
JNorth_Egg_Site_2014_df.shape

(510, 6)

In [255]:
JNorth_Egg_Site_2014_df = pd.DataFrame(JNorth_Egg_Site_2014_df)
JNorth_Egg_Site_2014_df.drop(['year'], axis=1, inplace=True)

In [256]:
JNorth_Egg_Site_2014_df.tail

<bound method NDFrame.tail of     sighting_date              species  latitude  longitude  number
430    2014-05-16  Monarch Egg Sighted    33.190    -94.356       1
431    2014-05-26  Monarch Egg Sighted    43.450    -79.683       2
432    2014-06-01  Monarch Egg Sighted    42.580    -87.895       1
433    2014-06-05  Monarch Egg Sighted    42.663    -85.599       1
434    2014-06-23  Monarch Egg Sighted    44.207    -87.603       1
..            ...                  ...       ...        ...     ...
935    2014-11-24  Monarch Egg Sighted    30.047    -95.214       3
936    2014-12-06  Monarch Egg Sighted    29.757    -95.365       1
937    2014-12-09  Monarch Egg Sighted    30.532    -88.171       1
938    2014-12-14  Monarch Egg Sighted    27.797    -97.399       1
939    2014-12-28  Monarch Egg Sighted    33.834   -118.313       1

[510 rows x 5 columns]>

In [257]:
JNorth_Egg_Site_2014_df['month'] = pd.DatetimeIndex(JNorth_Egg_Site_2014_df['sighting_date']).month

## January

In [258]:
JNorth_Egg_Site_2014_month_df =pd.DataFrame(JNorth_Egg_Site_2014_df)
JNorth_Egg_Site_2014_1_df = JNorth_Egg_Site_2014_month_df[JNorth_Egg_Site_2014_month_df['month']==1]
JNorth_Egg_Site_2014_1_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [259]:
JNorth_Egg_Site_2014_1_df.shape

(0, 6)

In [260]:
JNorth_Egg_Site_2014_1_df = pd.DataFrame(JNorth_Egg_Site_2014_1_df)
JNorth_Egg_Site_2014_1_df.drop(['month'], axis=1, inplace=True)

In [261]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2014_1_df['latitude'] = JNorth_Egg_Site_2014_1_df['latitude'].astype(float)
JNorth_Egg_Site_2014_1_df['longitude'] = JNorth_Egg_Site_2014_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [262]:
geojson = df_to_geojson(JNorth_Egg_Site_2014_1_df, cols)

In [263]:
file = open("JNorth_Egg_Site_2014_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [264]:
JNorth_Egg_Site_2014_month_df =pd.DataFrame(JNorth_Egg_Site_2014_df)
JNorth_Egg_Site_2014_2_df = JNorth_Egg_Site_2014_month_df[JNorth_Egg_Site_2014_month_df['month']==2]
JNorth_Egg_Site_2014_2_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [265]:
JNorth_Egg_Site_2014_2_df.shape

(0, 6)

In [266]:
JNorth_Egg_Site_2014_2_df = pd.DataFrame(JNorth_Egg_Site_2014_2_df)
JNorth_Egg_Site_2014_2_df.drop(['month'], axis=1, inplace=True)

In [267]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2014_2_df['latitude'] = JNorth_Egg_Site_2014_2_df['latitude'].astype(float)
JNorth_Egg_Site_2014_2_df['longitude'] = JNorth_Egg_Site_2014_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [268]:
geojson = df_to_geojson(JNorth_Egg_Site_2014_2_df, cols)

In [269]:
file = open("JNorth_Egg_Site_2014_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [270]:
JNorth_Egg_Site_2014_month_df =pd.DataFrame(JNorth_Egg_Site_2014_df)
JNorth_Egg_Site_2014_3_df = JNorth_Egg_Site_2014_month_df[JNorth_Egg_Site_2014_month_df['month']==3]
JNorth_Egg_Site_2014_3_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [271]:
JNorth_Egg_Site_2014_3_df.shape

(0, 6)

In [272]:
JNorth_Egg_Site_2014_3_df = pd.DataFrame(JNorth_Egg_Site_2014_3_df)
JNorth_Egg_Site_2014_3_df.drop(['month'], axis=1, inplace=True)

In [273]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2014_3_df['latitude'] = JNorth_Egg_Site_2014_3_df['latitude'].astype(float)
JNorth_Egg_Site_2014_3_df['longitude'] = JNorth_Egg_Site_2014_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [274]:
geojson = df_to_geojson(JNorth_Egg_Site_2014_3_df, cols)

In [275]:
file = open("JNorth_Egg_Site_2014_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [276]:
JNorth_Egg_Site_2014_month_df =pd.DataFrame(JNorth_Egg_Site_2014_df)
JNorth_Egg_Site_2014_4_df = JNorth_Egg_Site_2014_month_df[JNorth_Egg_Site_2014_month_df['month']==4]
JNorth_Egg_Site_2014_4_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [277]:
JNorth_Egg_Site_2014_4_df.shape

(0, 6)

In [278]:
JNorth_Egg_Site_2014_4_df = pd.DataFrame(JNorth_Egg_Site_2014_4_df)
JNorth_Egg_Site_2014_4_df.drop(['month'], axis=1, inplace=True)

In [279]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2014_4_df['latitude'] = JNorth_Egg_Site_2014_4_df['latitude'].astype(float)
JNorth_Egg_Site_2014_4_df['longitude'] = JNorth_Egg_Site_2014_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [280]:
geojson = df_to_geojson(JNorth_Egg_Site_2014_4_df, cols)

In [281]:
file = open("JNorth_Egg_Site_2014_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [282]:
JNorth_Egg_Site_2014_month_df =pd.DataFrame(JNorth_Egg_Site_2014_df)
JNorth_Egg_Site_2014_5_df = JNorth_Egg_Site_2014_month_df[JNorth_Egg_Site_2014_month_df['month']==5]
JNorth_Egg_Site_2014_5_df.tail()

,sighting_date,species,latitude,longitude,number,month
430,2014-05-16,Monarch Egg Sighted,33.19,-94.356,1,5
431,2014-05-26,Monarch Egg Sighted,43.45,-79.683,2,5


In [283]:
JNorth_Egg_Site_2014_5_df.shape

(2, 6)

In [284]:
JNorth_Egg_Site_2014_5_df = pd.DataFrame(JNorth_Egg_Site_2014_5_df)
JNorth_Egg_Site_2014_5_df.drop(['month'], axis=1, inplace=True)

In [285]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2014_5_df['latitude'] = JNorth_Egg_Site_2014_5_df['latitude'].astype(float)
JNorth_Egg_Site_2014_5_df['longitude'] = JNorth_Egg_Site_2014_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [286]:
geojson = df_to_geojson(JNorth_Egg_Site_2014_5_df, cols)

In [287]:
file = open("JNorth_Egg_Site_2014_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [288]:
JNorth_Egg_Site_2014_month_df =pd.DataFrame(JNorth_Egg_Site_2014_df)
JNorth_Egg_Site_2014_6_df = JNorth_Egg_Site_2014_month_df[JNorth_Egg_Site_2014_month_df['month']==6]
JNorth_Egg_Site_2014_6_df.tail()

,sighting_date,species,latitude,longitude,number,month
439,2014-06-28,Monarch Egg Sighted,42.305,-83.172,1,6
440,2014-06-29,Monarch Egg Sighted,42.950,-82.117,1,6
441,2014-06-29,Monarch Egg Sighted,46.867,-114.194,20,6
442,2014-06-30,Monarch Egg Sighted,41.789,-87.870,1,6
443,2014-06-30,Monarch Egg Sighted,44.986,-93.270,2,6


In [289]:
JNorth_Egg_Site_2014_6_df.shape

(12, 6)

In [290]:
JNorth_Egg_Site_2014_6_df = pd.DataFrame(JNorth_Egg_Site_2014_6_df)
JNorth_Egg_Site_2014_6_df.drop(['month'], axis=1, inplace=True)

In [291]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2014_6_df['latitude'] = JNorth_Egg_Site_2014_6_df['latitude'].astype(float)
JNorth_Egg_Site_2014_6_df['longitude'] = JNorth_Egg_Site_2014_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [292]:
geojson = df_to_geojson(JNorth_Egg_Site_2014_6_df, cols)

In [293]:
file = open("JNorth_Egg_Site_2014_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [294]:
JNorth_Egg_Site_2014_month_df =pd.DataFrame(JNorth_Egg_Site_2014_df)
JNorth_Egg_Site_2014_7_df = JNorth_Egg_Site_2014_month_df[JNorth_Egg_Site_2014_month_df['month']==7]
JNorth_Egg_Site_2014_7_df.tail()

,sighting_date,species,latitude,longitude,number,month
620,2014-07-31,Monarch Egg Sighted,40.446,-74.442,10,7
621,2014-07-31,Monarch Egg Sighted,40.810,-74.188,1,7
622,2014-07-31,Monarch Egg Sighted,41.808,-72.250,1,7
623,2014-07-31,Monarch Egg Sighted,41.845,-77.546,1,7
624,2014-07-31,Monarch Egg Sighted,44.706,-93.702,1,7


In [295]:
JNorth_Egg_Site_2014_7_df.shape

(181, 6)

In [296]:
JNorth_Egg_Site_2014_7_df = pd.DataFrame(JNorth_Egg_Site_2014_7_df)
JNorth_Egg_Site_2014_7_df.drop(['month'], axis=1, inplace=True)

In [297]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2014_7_df['latitude'] = JNorth_Egg_Site_2014_7_df['latitude'].astype(float)
JNorth_Egg_Site_2014_7_df['longitude'] = JNorth_Egg_Site_2014_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [298]:
geojson = df_to_geojson(JNorth_Egg_Site_2014_7_df, cols)

In [299]:
file = open("JNorth_Egg_Site_2014_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [300]:
JNorth_Egg_Site_2014_month_df =pd.DataFrame(JNorth_Egg_Site_2014_df)
JNorth_Egg_Site_2014_8_df = JNorth_Egg_Site_2014_month_df[JNorth_Egg_Site_2014_month_df['month']==8]
JNorth_Egg_Site_2014_8_df.tail()

,sighting_date,species,latitude,longitude,number,month
832,2014-08-30,Monarch Egg Sighted,43.872,-69.562,1,8
833,2014-08-31,Monarch Egg Sighted,29.470,-98.460,1,8
834,2014-08-31,Monarch Egg Sighted,32.645,-97.156,20,8
835,2014-08-31,Monarch Egg Sighted,36.278,-83.027,1,8
836,2014-08-31,Monarch Egg Sighted,44.986,-93.270,3,8


In [301]:
JNorth_Egg_Site_2014_8_df.shape

(212, 6)

In [302]:
JNorth_Egg_Site_2014_8_df= pd.DataFrame(JNorth_Egg_Site_2014_8_df)
JNorth_Egg_Site_2014_8_df.drop(['month'], axis=1, inplace=True)

In [303]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2014_8_df['latitude'] = JNorth_Egg_Site_2014_8_df['latitude'].astype(float)
JNorth_Egg_Site_2014_8_df['longitude'] = JNorth_Egg_Site_2014_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [304]:
geojson = df_to_geojson(JNorth_Egg_Site_2014_8_df, cols)

In [305]:
file = open("JNorth_Egg_Site_2014_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [306]:
JNorth_Egg_Site_2014_month_df =pd.DataFrame(JNorth_Egg_Site_2014_df)
JNorth_Egg_Site_2014_9_df = JNorth_Egg_Site_2014_month_df[JNorth_Egg_Site_2014_month_df['month']==9]
JNorth_Egg_Site_2014_9_df.tail()

,sighting_date,species,latitude,longitude,number,month
908,2014-09-26,Monarch Egg Sighted,30.603,-99.473,1,9
909,2014-09-28,Monarch Egg Sighted,32.774,-91.709,1,9
910,2014-09-30,Monarch Egg Sighted,29.984,-90.166,1,9
911,2014-09-30,Monarch Egg Sighted,30.449,-91.188,1,9
912,2014-09-30,Monarch Egg Sighted,32.131,-97.855,1,9


In [307]:
JNorth_Egg_Site_2014_9_df.shape

(76, 6)

In [308]:
JNorth_Egg_Site_2014_9_df = pd.DataFrame(JNorth_Egg_Site_2014_9_df)
JNorth_Egg_Site_2014_9_df.drop(['month'], axis=1, inplace=True)

In [309]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2014_9_df['latitude'] = JNorth_Egg_Site_2014_9_df['latitude'].astype(float)
JNorth_Egg_Site_2014_9_df['longitude'] = JNorth_Egg_Site_2014_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [310]:
geojson = df_to_geojson(JNorth_Egg_Site_2014_9_df, cols)

In [311]:
file = open("JNorth_Egg_Site_2014_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [312]:
JNorth_Egg_Site_2014_month_df =pd.DataFrame(JNorth_Egg_Site_2014_df)
JNorth_Egg_Site_2014_10_df = JNorth_Egg_Site_2014_month_df[JNorth_Egg_Site_2014_month_df['month']==10]
JNorth_Egg_Site_2014_10_df.tail()

,sighting_date,species,latitude,longitude,number,month
926,2014-10-24,Monarch Egg Sighted,30.389,-89.098,4,10
927,2014-10-26,Monarch Egg Sighted,25.440,-102.179,1,10
928,2014-10-26,Monarch Egg Sighted,30.061,-95.384,1,10
929,2014-10-28,Monarch Egg Sighted,33.090,-96.640,1,10
930,2014-10-29,Monarch Egg Sighted,30.230,-97.714,1,10


In [313]:
JNorth_Egg_Site_2014_10_df.shape

(18, 6)

In [314]:
JNorth_Egg_Site_2014_10_df = pd.DataFrame(JNorth_Egg_Site_2014_10_df)
JNorth_Egg_Site_2014_10_df.drop(['month'], axis=1, inplace=True)

In [315]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2014_10_df['latitude'] = JNorth_Egg_Site_2014_10_df['latitude'].astype(float)
JNorth_Egg_Site_2014_10_df['longitude'] = JNorth_Egg_Site_2014_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [316]:
geojson = df_to_geojson(JNorth_Egg_Site_2014_10_df, cols)

In [317]:
file = open("JNorth_Egg_Site_2014_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [318]:
JNorth_Egg_Site_2014_month_df =pd.DataFrame(JNorth_Egg_Site_2014_df)
JNorth_Egg_Site_2014_11_df = JNorth_Egg_Site_2014_month_df[JNorth_Egg_Site_2014_month_df['month']==11]
JNorth_Egg_Site_2014_11_df.tail()

,sighting_date,species,latitude,longitude,number,month
931,2014-11-07,Monarch Egg Sighted,29.757,-95.365,1,11
932,2014-11-10,Monarch Egg Sighted,29.757,-95.365,1,11
933,2014-11-11,Monarch Egg Sighted,37.936,-122.536,1,11
934,2014-11-17,Monarch Egg Sighted,29.757,-95.365,1,11
935,2014-11-24,Monarch Egg Sighted,30.047,-95.214,3,11


In [319]:
JNorth_Egg_Site_2014_11_df.shape

(5, 6)

In [320]:
JNorth_Egg_Site_2014_11_df = pd.DataFrame(JNorth_Egg_Site_2014_11_df)
JNorth_Egg_Site_2014_11_df.drop(['month'], axis=1, inplace=True)

In [321]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2014_11_df['latitude'] = JNorth_Egg_Site_2014_11_df['latitude'].astype(float)
JNorth_Egg_Site_2014_11_df['longitude'] = JNorth_Egg_Site_2014_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [322]:
geojson = df_to_geojson(JNorth_Egg_Site_2014_11_df, cols)

In [323]:
file = open("JNorth_Egg_Site_2014_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [324]:
JNorth_Egg_Site_2014_month_df =pd.DataFrame(JNorth_Egg_Site_2014_df)
JNorth_Egg_Site_2014_12_df = JNorth_Egg_Site_2014_month_df[JNorth_Egg_Site_2014_month_df['month']==12]
JNorth_Egg_Site_2014_12_df.tail()

,sighting_date,species,latitude,longitude,number,month
936,2014-12-06,Monarch Egg Sighted,29.757,-95.365,1,12
937,2014-12-09,Monarch Egg Sighted,30.532,-88.171,1,12
938,2014-12-14,Monarch Egg Sighted,27.797,-97.399,1,12
939,2014-12-28,Monarch Egg Sighted,33.834,-118.313,1,12


In [325]:
JNorth_Egg_Site_2014_12_df.shape

(4, 6)

In [326]:
JNorth_Egg_Site_2014_12_df = pd.DataFrame(JNorth_Egg_Site_2014_12_df)
JNorth_Egg_Site_2014_12_df.drop(['month'], axis=1, inplace=True)

In [327]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2014_12_df['latitude'] = JNorth_Egg_Site_2014_12_df['latitude'].astype(float)
JNorth_Egg_Site_2014_12_df['longitude'] = JNorth_Egg_Site_2014_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [328]:
geojson = df_to_geojson(JNorth_Egg_Site_2014_12_df, cols)

In [329]:
file = open("JNorth_Egg_Site_2014_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# 2015

In [330]:
JNorth_Egg_Site_2015_df = JNorth_Egg_Site_df[JNorth_Egg_Site_df['year']==2015]
JNorth_Egg_Site_2015_df.tail()

,sighting_date,species,latitude,longitude,number,year
1671,2015-12-12,Monarch Egg Sighted,33.902,-117.606,25,2015
1672,2015-12-17,Monarch Egg Sighted,29.962,-90.218,11,2015
1673,2015-12-22,Monarch Egg Sighted,28.910,-96.801,50,2015
1674,2015-12-27,Monarch Egg Sighted,28.487,-81.605,6,2015
1675,2015-12-27,Monarch Egg Sighted,29.835,-81.277,20,2015


In [331]:
JNorth_Egg_Site_2015_df.shape

(736, 6)

In [332]:
JNorth_Egg_Site_2015_df = pd.DataFrame(JNorth_Egg_Site_2015_df)
JNorth_Egg_Site_2015_df.drop(['year'], axis=1, inplace=True)

In [333]:
JNorth_Egg_Site_2015_df.tail

<bound method NDFrame.tail of      sighting_date              species  latitude  longitude  number
940     2015-01-17  Monarch Egg Sighted    31.233    -89.773      10
941     2015-05-02  Monarch Egg Sighted    39.982    -74.161       6
942     2015-05-07  Monarch Egg Sighted    40.422    -80.561      33
943     2015-06-12  Monarch Egg Sighted    44.986    -93.270       2
944     2015-06-25  Monarch Egg Sighted    41.679   -111.849      11
...            ...                  ...       ...        ...     ...
1671    2015-12-12  Monarch Egg Sighted    33.902   -117.606      25
1672    2015-12-17  Monarch Egg Sighted    29.962    -90.218      11
1673    2015-12-22  Monarch Egg Sighted    28.910    -96.801      50
1674    2015-12-27  Monarch Egg Sighted    28.487    -81.605       6
1675    2015-12-27  Monarch Egg Sighted    29.835    -81.277      20

[736 rows x 5 columns]>

In [334]:
JNorth_Egg_Site_2015_df['month'] = pd.DatetimeIndex(JNorth_Egg_Site_2015_df['sighting_date']).month

## January

In [335]:
JNorth_Egg_Site_2015_month_df =pd.DataFrame(JNorth_Egg_Site_2015_df)
JNorth_Egg_Site_2015_1_df = JNorth_Egg_Site_2015_month_df[JNorth_Egg_Site_2015_month_df['month']==1]
JNorth_Egg_Site_2015_1_df.tail()

,sighting_date,species,latitude,longitude,number,month
940,2015-01-17,Monarch Egg Sighted,31.233,-89.773,10,1


In [336]:
JNorth_Egg_Site_2015_1_df.shape

(1, 6)

In [337]:
JNorth_Egg_Site_2015_1_df = pd.DataFrame(JNorth_Egg_Site_2015_1_df)
JNorth_Egg_Site_2015_1_df.drop(['month'], axis=1, inplace=True)

In [338]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2015_1_df['latitude'] = JNorth_Egg_Site_2015_1_df['latitude'].astype(float)
JNorth_Egg_Site_2015_1_df['longitude'] = JNorth_Egg_Site_2015_1_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [339]:
geojson = df_to_geojson(JNorth_Egg_Site_2015_1_df, cols)

In [340]:
file = open("JNorth_Egg_Site_2015_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# February

In [341]:
JNorth_Egg_Site_2015_month_df =pd.DataFrame(JNorth_Egg_Site_2015_df)
JNorth_Egg_Site_2015_2_df = JNorth_Egg_Site_2015_month_df[JNorth_Egg_Site_2015_month_df['month']==2]
JNorth_Egg_Site_2015_2_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [342]:
JNorth_Egg_Site_2015_2_df.shape

(0, 6)

In [343]:
JNorth_Egg_Site_2015_2_df = pd.DataFrame(JNorth_Egg_Site_2015_2_df)
JNorth_Egg_Site_2015_2_df.drop(['month'], axis=1, inplace=True)

In [344]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2015_2_df['latitude'] = JNorth_Egg_Site_2015_2_df['latitude'].astype(float)
JNorth_Egg_Site_2015_2_df['longitude'] = JNorth_Egg_Site_2015_2_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [345]:
geojson = df_to_geojson(JNorth_Egg_Site_2015_2_df, cols)

In [346]:
file = open("JNorth_Egg_Site_2015_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# March

In [347]:
JNorth_Egg_Site_2015_month_df =pd.DataFrame(JNorth_Egg_Site_2015_df)
JNorth_Egg_Site_2015_3_df = JNorth_Egg_Site_2015_month_df[JNorth_Egg_Site_2015_month_df['month']==3]
JNorth_Egg_Site_2015_3_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [348]:
JNorth_Egg_Site_2015_3_df.shape

(0, 6)

In [349]:
JNorth_Egg_Site_2015_3_df = pd.DataFrame(JNorth_Egg_Site_2015_3_df)
JNorth_Egg_Site_2015_3_df.drop(['month'], axis=1, inplace=True)

In [350]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2015_3_df['latitude'] = JNorth_Egg_Site_2015_3_df['latitude'].astype(float)
JNorth_Egg_Site_2015_3_df['longitude'] = JNorth_Egg_Site_2015_3_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [351]:
geojson = df_to_geojson(JNorth_Egg_Site_2015_3_df, cols)

In [352]:
file = open("JNorth_Egg_Site_2015_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# April

In [353]:
JNorth_Egg_Site_2015_month_df =pd.DataFrame(JNorth_Egg_Site_2015_df)
JNorth_Egg_Site_2015_4_df = JNorth_Egg_Site_2015_month_df[JNorth_Egg_Site_2015_month_df['month']==4]
JNorth_Egg_Site_2015_4_df.tail()

,sighting_date,species,latitude,longitude,number,month


In [354]:
JNorth_Egg_Site_2015_4_df.shape

(0, 6)

In [355]:
JNorth_Egg_Site_2015_4_df= pd.DataFrame(JNorth_Egg_Site_2015_4_df)
JNorth_Egg_Site_2015_4_df.drop(['month'], axis=1, inplace=True)

In [356]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2015_4_df['latitude'] = JNorth_Egg_Site_2015_4_df['latitude'].astype(float)
JNorth_Egg_Site_2015_4_df['longitude'] = JNorth_Egg_Site_2015_4_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [357]:
geojson = df_to_geojson(JNorth_Egg_Site_2015_4_df, cols)

In [358]:
file = open("JNorth_Egg_Site_2015_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# May

In [359]:
JNorth_Egg_Site_2015_month_df =pd.DataFrame(JNorth_Egg_Site_2015_df)
JNorth_Egg_Site_2015_5_df = JNorth_Egg_Site_2015_month_df[JNorth_Egg_Site_2015_month_df['month']==5]
JNorth_Egg_Site_2015_5_df.tail()

,sighting_date,species,latitude,longitude,number,month
941,2015-05-02,Monarch Egg Sighted,39.982,-74.161,6,5
942,2015-05-07,Monarch Egg Sighted,40.422,-80.561,33,5


In [360]:
JNorth_Egg_Site_2015_5_df.shape

(2, 6)

In [361]:
JNorth_Egg_Site_2015_5_df = pd.DataFrame(JNorth_Egg_Site_2015_5_df)
JNorth_Egg_Site_2015_5_df.drop(['month'], axis=1, inplace=True)

In [362]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2015_5_df['latitude'] = JNorth_Egg_Site_2015_5_df['latitude'].astype(float)
JNorth_Egg_Site_2015_5_df['longitude'] = JNorth_Egg_Site_2015_5_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [363]:
geojson = df_to_geojson(JNorth_Egg_Site_2015_5_df, cols)

In [364]:
file = open("JNorth_Egg_Site_2015_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# June

In [365]:
JNorth_Egg_Site_2015_month_df =pd.DataFrame(JNorth_Egg_Site_2015_df)
JNorth_Egg_Site_2015_6_df = JNorth_Egg_Site_2015_month_df[JNorth_Egg_Site_2015_month_df['month']==6]
JNorth_Egg_Site_2015_6_df.tail()

,sighting_date,species,latitude,longitude,number,month
943,2015-06-12,Monarch Egg Sighted,44.986,-93.270,2,6
944,2015-06-25,Monarch Egg Sighted,41.679,-111.849,11,6


In [366]:
JNorth_Egg_Site_2015_6_df.shape

(2, 6)

In [367]:
JNorth_Egg_Site_2015_6_df = pd.DataFrame(JNorth_Egg_Site_2015_6_df)
JNorth_Egg_Site_2015_6_df.drop(['month'], axis=1, inplace=True)

In [368]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2015_6_df['latitude'] = JNorth_Egg_Site_2015_6_df['latitude'].astype(float)
JNorth_Egg_Site_2015_6_df['longitude'] = JNorth_Egg_Site_2015_6_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [369]:
geojson = df_to_geojson(JNorth_Egg_Site_2015_6_df, cols)

In [370]:
file = open("JNorth_Egg_Site_2015_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# July

In [371]:
JNorth_Egg_Site_2015_month_df =pd.DataFrame(JNorth_Egg_Site_2015_df)
JNorth_Egg_Site_2015_7_df = JNorth_Egg_Site_2015_month_df[JNorth_Egg_Site_2015_month_df['month']==7]
JNorth_Egg_Site_2015_7_df.tail()

,sighting_date,species,latitude,longitude,number,month
1078,2015-07-31,Monarch Egg Sighted,42.400,-82.183,2,7
1079,2015-07-31,Monarch Egg Sighted,42.446,-71.231,13,7
1080,2015-07-31,Monarch Egg Sighted,42.900,-79.233,1,7
1081,2015-07-31,Monarch Egg Sighted,43.750,-72.370,5,7
1082,2015-07-31,Monarch Egg Sighted,44.050,-79.467,3,7


In [372]:
JNorth_Egg_Site_2015_7_df.shape

(138, 6)

In [373]:
JNorth_Egg_Site_2015_7_df = pd.DataFrame(JNorth_Egg_Site_2015_7_df)
JNorth_Egg_Site_2015_7_df.drop(['month'], axis=1, inplace=True)

In [374]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2015_7_df['latitude'] = JNorth_Egg_Site_2015_7_df['latitude'].astype(float)
JNorth_Egg_Site_2015_7_df['longitude'] = JNorth_Egg_Site_2015_7_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [375]:
geojson = df_to_geojson(JNorth_Egg_Site_2015_7_df, cols)

In [376]:
file = open("JNorth_Egg_Site_2015_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# August

In [377]:
JNorth_Egg_Site_2015_month_df =pd.DataFrame(JNorth_Egg_Site_2015_df)
JNorth_Egg_Site_2015_8_df = JNorth_Egg_Site_2015_month_df[JNorth_Egg_Site_2015_month_df['month']==8]
JNorth_Egg_Site_2015_8_df.tail()

,sighting_date,species,latitude,longitude,number,month
1440,2015-08-31,Monarch Egg Sighted,30.449,-91.188,30,8
1441,2015-08-31,Monarch Egg Sighted,39.138,-77.294,24,8
1442,2015-08-31,Monarch Egg Sighted,39.429,-81.495,3,8
1443,2015-08-31,Monarch Egg Sighted,40.183,-88.380,12,8
1444,2015-08-31,Monarch Egg Sighted,40.889,-74.046,15,8


In [378]:
JNorth_Egg_Site_2015_8_df.shape

(362, 6)

In [379]:
JNorth_Egg_Site_2015_8_df = pd.DataFrame(JNorth_Egg_Site_2015_8_df)
JNorth_Egg_Site_2015_8_df.drop(['month'], axis=1, inplace=True)

In [380]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2015_8_df['latitude'] = JNorth_Egg_Site_2015_8_df['latitude'].astype(float)
JNorth_Egg_Site_2015_8_df['longitude'] = JNorth_Egg_Site_2015_8_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [381]:
geojson = df_to_geojson(JNorth_Egg_Site_2015_8_df, cols)

In [382]:
file = open("JNorth_Egg_Site_2015_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# September

In [383]:
JNorth_Egg_Site_2015_month_df =pd.DataFrame(JNorth_Egg_Site_2015_df)
JNorth_Egg_Site_2015_9_df = JNorth_Egg_Site_2015_month_df[JNorth_Egg_Site_2015_month_df['month']==9]
JNorth_Egg_Site_2015_9_df.tail()

,sighting_date,species,latitude,longitude,number,month
1602,2015-09-28,Monarch Egg Sighted,36.297,-95.785,190,9
1603,2015-09-29,Monarch Egg Sighted,30.315,-98.914,200,9
1604,2015-09-29,Monarch Egg Sighted,34.138,-118.166,6,9
1605,2015-09-30,Monarch Egg Sighted,31.087,-97.324,1,9
1606,2015-09-30,Monarch Egg Sighted,31.140,-94.355,3,9


In [384]:
JNorth_Egg_Site_2015_9_df.shape

(162, 6)

In [385]:
JNorth_Egg_Site_2015_9_df = pd.DataFrame(JNorth_Egg_Site_2015_9_df)
JNorth_Egg_Site_2015_9_df.drop(['month'], axis=1, inplace=True)

In [386]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2015_9_df['latitude'] = JNorth_Egg_Site_2015_9_df['latitude'].astype(float)
JNorth_Egg_Site_2015_9_df['longitude'] = JNorth_Egg_Site_2015_9_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [387]:
geojson = df_to_geojson(JNorth_Egg_Site_2015_9_df, cols)

In [388]:
file = open("JNorth_Egg_Site_2015_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# October

In [389]:
JNorth_Egg_Site_2015_month_df =pd.DataFrame(JNorth_Egg_Site_2015_df)
JNorth_Egg_Site_2015_10_df = JNorth_Egg_Site_2015_month_df[JNorth_Egg_Site_2015_month_df['month']==10]
JNorth_Egg_Site_2015_10_df.tail()

,sighting_date,species,latitude,longitude,number,month
1644,2015-10-24,Monarch Egg Sighted,37.676,-122.134,6,10
1645,2015-10-25,Monarch Egg Sighted,29.777,-94.890,50,10
1646,2015-10-26,Monarch Egg Sighted,29.957,-90.078,4,10
1647,2015-10-27,Monarch Egg Sighted,34.212,-118.230,2,10
1648,2015-10-30,Monarch Egg Sighted,31.233,-89.773,20,10


In [390]:
JNorth_Egg_Site_2015_10_df.shape

(42, 6)

In [391]:
JNorth_Egg_Site_2015_10_df = pd.DataFrame(JNorth_Egg_Site_2015_10_df)
JNorth_Egg_Site_2015_10_df.drop(['month'], axis=1, inplace=True)

In [392]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2015_10_df['latitude'] = JNorth_Egg_Site_2015_10_df['latitude'].astype(float)
JNorth_Egg_Site_2015_10_df['longitude'] = JNorth_Egg_Site_2015_10_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [393]:
geojson = df_to_geojson(JNorth_Egg_Site_2015_10_df, cols)

In [394]:
file = open("JNorth_Egg_Site_2015_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# November

In [395]:
JNorth_Egg_Site_2015_month_df =pd.DataFrame(JNorth_Egg_Site_2015_df)
JNorth_Egg_Site_2015_11_df = JNorth_Egg_Site_2015_month_df[JNorth_Egg_Site_2015_month_df['month']==11]
JNorth_Egg_Site_2015_11_df.tail()

,sighting_date,species,latitude,longitude,number,month
1663,2015-11-17,Monarch Egg Sighted,33.151,-117.303,11,11
1664,2015-11-20,Monarch Egg Sighted,29.545,-81.224,42,11
1665,2015-11-21,Monarch Egg Sighted,33.994,-118.462,10,11
1666,2015-11-23,Monarch Egg Sighted,29.459,-98.457,2,11
1667,2015-11-30,Monarch Egg Sighted,29.705,-95.461,21,11


In [396]:
JNorth_Egg_Site_2015_11_df.shape

(19, 6)

In [397]:
JNorth_Egg_Site_2015_11_df = pd.DataFrame(JNorth_Egg_Site_2015_11_df)
JNorth_Egg_Site_2015_11_df.drop(['month'], axis=1, inplace=True)

In [398]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2015_11_df['latitude'] = JNorth_Egg_Site_2015_11_df['latitude'].astype(float)
JNorth_Egg_Site_2015_11_df['longitude'] = JNorth_Egg_Site_2015_11_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [399]:
geojson = df_to_geojson(JNorth_Egg_Site_2015_11_df, cols)

In [400]:
file = open("JNorth_Egg_Site_2015_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

# December

In [401]:
JNorth_Egg_Site_2015_month_df =pd.DataFrame(JNorth_Egg_Site_2015_df)
JNorth_Egg_Site_2015_12_df = JNorth_Egg_Site_2015_month_df[JNorth_Egg_Site_2015_month_df['month']==12]
JNorth_Egg_Site_2015_12_df.tail()

,sighting_date,species,latitude,longitude,number,month
1671,2015-12-12,Monarch Egg Sighted,33.902,-117.606,25,12
1672,2015-12-17,Monarch Egg Sighted,29.962,-90.218,11,12
1673,2015-12-22,Monarch Egg Sighted,28.910,-96.801,50,12
1674,2015-12-27,Monarch Egg Sighted,28.487,-81.605,6,12
1675,2015-12-27,Monarch Egg Sighted,29.835,-81.277,20,12


In [402]:
JNorth_Egg_Site_2015_12_df.shape

(8, 6)

In [403]:
JNorth_Egg_Site_2015_12_df = pd.DataFrame(JNorth_Egg_Site_2015_12_df)
JNorth_Egg_Site_2015_12_df.drop(['month'], axis=1, inplace=True)

In [404]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
JNorth_Egg_Site_2015_12_df['latitude'] = JNorth_Egg_Site_2015_12_df['latitude'].astype(float)
JNorth_Egg_Site_2015_12_df['longitude'] = JNorth_Egg_Site_2015_12_df['longitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [405]:
geojson = df_to_geojson(JNorth_Egg_Site_2015_12_df, cols)

In [406]:
file = open("JNorth_Egg_Site_2015_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()